In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros):
    error  = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 행렬 R 생성
    full_pred_matrix = np.dot(P,Q.T)
    
    # 실제 R 행렬에서 null이 아닌 값의 위치 인덱스를 추출하여 실제 R 행렬과 예측 행렬의 rmse 추출
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    R_non_zeros = R[x_non_zero_ind,y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind,y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    
    rmse = np.sqrt(mse)
    
    return rmse

In [8]:
def matrix_factorization(R,K,steps=200,learning_rate = 0.01,r_lambda=0.01):
    num_users, num_items = R.shape
    np.random.seed(1)
    
    P = np.random.normal(scale=1./K, size = (num_users,K))
    Q = np.random.normal(scale=1./K, size = (num_items,K))
    prev_rmse = 10000
    break_count = 0
    
    # R > 0인 행 위치, 열 위치, 값을  non_zeros 리스트에 객체 저장
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    
    for step in range(steps): # steps는 SGD의 반복횟수
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류값 구함
            eij = r - np.dot(P[i,:],Q[j,:].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:]=P[i,:] + learning_rate*(eij*Q[j,:] - r_lambda*P[i,:])
            Q[j,:]=Q[j,:] + learning_rate*(eij*P[i,:] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q,non_zeros)
        if (step%10)==0: # 10회 반복시 오류 출력
            print(f'iteration step:{step}, rmse:{rmse}')
    return P,Q

In [9]:
import pandas as pd
import numpy as np

movies = pd.read_csv('./dataset/ml-latest-small/movies.csv')
ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId',columns='movieId')

rating_movies = pd.merge(ratings, movies, on='movieId')

ratings_matrix = rating_movies.pivot_table('rating',index='userId',columns = 'title')


In [10]:
P,Q = matrix_factorization(ratings_matrix.values, K = 50, steps = 200, learning_rate = 0.01,r_lambda = 0.01)

pred_matrix = np.dot(P,Q.T)

iteration step:0, rmse:2.9023619751336867
iteration step:10, rmse:0.7335768591017927
iteration step:20, rmse:0.5115539026853442
iteration step:30, rmse:0.37261628282537446
iteration step:40, rmse:0.2960818299181014
iteration step:50, rmse:0.2520353192341642
iteration step:60, rmse:0.22487503275269854
iteration step:70, rmse:0.2068545530233154
iteration step:80, rmse:0.19413418783028688
iteration step:90, rmse:0.18470082002720403
iteration step:100, rmse:0.17742927527209104
iteration step:110, rmse:0.1716522696470749
iteration step:120, rmse:0.16695181946871726
iteration step:130, rmse:0.16305292191997545
iteration step:140, rmse:0.15976691929679646
iteration step:150, rmse:0.15695986999457318
iteration step:160, rmse:0.1545339818671543
iteration step:170, rmse:0.15241618551077643
iteration step:180, rmse:0.1505508073962831
iteration step:190, rmse:0.14889470913232092


In [11]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix,index = ratings_matrix.index,columns = ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941
4,2.628629,3.035550,2.575746,3.706912,3.430636,0.706441,3.330280,1.978826,4.560368,2.775710,...,1.046116,2.912178,2.479592,2.231915,1.888629,2.211364,0.645603,1.585734,3.542892,0.591540
5,2.116148,3.084761,2.747679,3.783490,3.946990,0.883259,1.958953,1.757317,2.054312,2.775258,...,0.956159,3.893975,2.717024,2.002443,2.053337,3.983639,2.099626,1.423718,2.490428,0.531403


In [15]:
ratings_pred_matrix.loc[ratings_pred_matrix.index == 9]

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
9,2.315852,3.310448,2.93207,3.676525,3.637459,0.834727,2.729853,1.797212,3.435542,3.11855,...,1.081084,2.985101,2.830357,2.07256,1.935708,3.53766,1.097133,1.8195,2.805732,0.717371


In [12]:
def get_unseen_movies(ratings_matrix,userId):
    user_rating = ratings_matrix.loc[userId,:]
    already_seen = user_rating[user_rating>0].index.tolist()
    
    movies_list =ratings_matrix.columns.tolist()
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

def recomm_movie_by_userId(pred_df,userId,unseen_list,top_n=10):
    recomm_movies = pred_df.loc[userId,unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [13]:
unseen_list = get_unseen_movies(ratings_matrix, 9)

recomm_movies = recomm_movie_by_userId(ratings_pred_matrix, 9, unseen_list, top_n = 10)

recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns = ['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690
